<a href="https://colab.research.google.com/github/rigonzal/ANN/blob/master/ANN/Approach2/Pregunta_3_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INF-395 Redes Neuronales Artificiales II-2018

## Tarea 2 - Redes Convolucionales y sus aplicaciones


26/11/2018

* Rodrigo González Smith 201303026-2
* Ignacio Valenzuela Albornoz 2014073055-1

## Introducción Parte 3



Débido al tema del trabajo usaremos **Keras** para la creación de las redes y su posterior entrenamiento/validación. Como también las librerías comunes de trabajo de Python que nos facilitarán nuestras tareas experimentales:

In [0]:
#Correr solo en Colab
!pip install -q keras
!pip install -q nltk

In [0]:
import numpy as np   
import os, re, sys
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, rmsprop
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
import h5py
import nltk
nltk.download()

%matplotlib inline

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Unzipping corpora/stopwords.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q




---


## 1) CNN sobre texto

Ahora usaremos redes neuronales convolucionales para (NLP), es decir, procesamiento de Lenguaje Natural. El dataset cuenta con miles de avisos de trabajo, y el objetivo es a partir de estos poder predecir el salario asociado. Como bien dice el [sitio oficial](https://www.kaggle.com/c/job-salary-prediction)  que entrega el dataset, es crear un *Salary prediction engine*. 

In [15]:

stoplist = stopwords.words('english')
# lemmatizer (WordNetLemmatizer())
# stemming?
# SUBIR ARCHIVO A COLAB
from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [37]:
import csv
df=pd.read_csv("Train_rev1.csv",quoting=3, encoding='utf-8')
df.columns

ParserError: ignored

In [0]:


def pre_procesar(df):
    #preprocesar texto de los anuncios
    #Eliminación de stopwords, stemming/lemmatization, puntuación, etc
    for s in textos:
        s= s.lower()
        s= re.sub(r'[^\w]', ' ',s)
        s= re.sub(r'\b[a-z]\b', ' ',  s)
        s= re.sub(r'\b[a-z][a-z]\b', ' ',  s)
        s=re.sub(r'\b[0-9]\b', ' ',  s)
        s=re.sub(r'\b[0-9][0-9]\b', ' ',  s)
        s=re.sub(r'\b[0-9][0-9][0-9]\b', ' ',  s)
        s= re.sub(r'[^\w.]', ' ', s)
        s= list(s.split())
        #s= eliminación de stopwords
        #s= stemming o lemmatization
        #guardar texto procesado

    return df_procesado 
y_dataframe=df['SalaryNormalized'].values
x_dataframe=df[['FullDescription',...]]

k=len(df_procesado)
x_train=df_procesado[0:int(k*0.70)] #70% training
x_val=df_procesado[int(k*0.70):int(k*0.85)] #15% validation
x_test=df_procesado[int(k*0.85):] #15% test